In [3]:
!kaggle competitions download -c cifar-10

/bin/bash: line 1: kaggle: command not found


In [ ]:
import numpy as np 
import pandas as pd 
import py7zr

import os
for dirname, _, filenames in os.walk('kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

kaggle/input/cifar-10.zip
kaggle/input/cifar-10/trainLabels.csv:Zone.Identifier
kaggle/input/cifar-10/test.7z:Zone.Identifier
kaggle/input/cifar-10/test.7z
kaggle/input/cifar-10/train.7z:Zone.Identifier
kaggle/input/cifar-10/sampleSubmission.csv
kaggle/input/cifar-10/trainLabels.csv
kaggle/input/cifar-10/train.7z
kaggle/input/cifar-10/sampleSubmission.csv:Zone.Identifier


In [ ]:
# # Extracting the cifar-10 archive


# archive_path = 'kaggle/input/cifar-10/train.7z'
# extract_folder = 'kaggle/temp'

# with py7zr.SevenZipFile(archive_path, mode='r') as archive:
#     archive.extractall(path=extract_folder)

In [ ]:
# Create directories for each class in train and validation folders

dataset_path_train = 'kaggle/temp/train'
dataset_path_valid = 'kaggle/temp/valid'

classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

# for label in classes:
#     train_path = os.path.join(dataset_path_train, label)
#     os.makedirs(train_path, exist_ok=True)
    
#     valid_path = os.path.join(dataset_path_valid, label)
#     os.makedirs(valid_path, exist_ok=True)

In [ ]:
# # Randomly assign each image to train (80%) or validation (20%) set

train_labels = pd.read_csv('kaggle/input/cifar-10/trainLabels.csv')

# # Iterate over each label in the CSV    
# for (int_ind, row) in train_labels.iterrows():
#     # Construct the image file name
#     id = str(row["id"]) + ".png"
#     source_path = os.path.join(dataset_path_train, id)
    
#     # Check if the image file exists
#     if not os.path.exists(source_path):
#         print(f"Missing source File: {source_path}")
#         continue
        
#     # Randomly assign the image to train (80%) or validation (20%) set
#     p = np.random.random()
#     if p <= 0.8:
#         target_path = os.path.join(dataset_path_train, row["label"], id)
#     else:
#         target_path = os.path.join(dataset_path_valid, row["label"], id)
    
#     # Move the image file to the corresponding target directory
#     os.replace(source_path, target_path)


In [27]:


# Define the path to the 7-zip archive containing the test data
archive_path = 'kaggle/input/cifar-10/test.7z'

# Define the directory where the contents of the archive will be extracted
extract_folder = 'kaggle/temp/test'

# # Open the 7-zip archive in read mode
# with py7zr.SevenZipFile(archive_path, mode='r') as archive:
#     # Extract all the contents of the archive to the specified directory
#     archive.extractall(path=extract_folder)



In [28]:


# Get the number of files in the test directory
len(os.listdir('kaggle/temp/test/test'))

# Define the path to the test directory
path = 'kaggle/temp/test/test'

# List all files in the test directory
f = os.listdir(path)

# Create a new list with the full path of each file
new = [str(path) + '/' + s for s in f]



In [ ]:

from fastai.vision.all import ImageDataLoaders, Resize, aug_transforms, Normalize, imagenet_stats
# Define the Image Data Loaders
dls = ImageDataLoaders.from_folder(
    path='kaggle/temp',              # Path to the folder containing 'train' and 'valid' subdirectories
    train='train',                    # Name of the training data subdirectory
    valid='valid',                    # Name of the validation data subdirectory
    item_tfms=Resize(224),            # Resize all images to 224x224 pixels
    bs=64,                            # Batch size of 64
    batch_tfms=[*aug_transforms(),    # Apply data augmentation transformations
                Normalize.from_stats(*imagenet_stats)]  # Normalize images using ImageNet statistics
)

In [ ]:
from fastai.vision.all import vision_learner, resnet152, error_rate
# Create a learner with a pre-trained ResNet152 model
learn = vision_learner(dls, resnet152, metrics=error_rate)

# Fine-tune the model for 10 epochs
learn.fine_tune(5)  

epoch,train_loss,valid_loss,error_rate,time


: 

In [ ]:
# Create a test DataLoader from the list of file paths
test_dl = learn.dls.test_dl(new)

# Get predictions for the test data
class_score, y = learn.get_preds(dl=test_dl)

# Get the predicted class indices
class_score = np.argmax(class_score, axis=1)

# Get the prediction for the second test image (index 1)
class_score[1].item()

# Convert the predicted class indices to a list
classScore = class_score.tolist()

# Get the vocabulary of classes
learn.dls.vocab

# Define a dictionary mapping class indices to class names
classes = {0: 'airplane', 1: 'automobile', 2: 'bird', 3: 'cat', 4: 'deer', 5: 'dog', 6: 'frog', 7: 'horse', 8: 'ship', 9: 'truck'}

# Initialize an empty array to store the predicted classes
predicted_classes = np.empty(shape=300000, dtype=np.dtype('U20'))

In [ ]:


# Initialize an index variable
index = 0

# Loop through the list of predicted class indices
for i in classScore:
    # Assign the class name corresponding to the class index to the predicted_classes array
    predicted_classes[index] = classes[i]
    
    # Increment the index for the next iteration
    index += 1



In [ ]:


# Define the directory containing the test images
directory = 'kaggle/temp/test/test'

# Extract image IDs by filtering out non-digit characters from the file names
ImageId = [ (''.join(filter(str.isdigit, name))) for name in os.listdir(directory)]

# Create a DataFrame for the submission with 'id' and 'label' columns
submission = pd.DataFrame({
    "id": ImageId,          # Assign the extracted image IDs to the 'id' column
    "label": predicted_classes  # Assign the predicted class names to the 'label' column
})

# Convert the 'id' column to integers
submission['id'] = submission['id'].astype(int)

# Save the DataFrame to a CSV file for submission, without the index
submission.to_csv('submission.csv', index=False)

